In [1]:
from utils.data_management import load_data
import config as cfg

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os

2021-07-15 21:42:59.398547: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


# Chargement des données

## Chargement des données

In [2]:
# On profite de la puissance de Python : Les dictionnaires
datasets_map = {
    'cifar': None,
    'mnist': None,
    'fashion': None
}

for name in datasets_map.keys():
    datasets_map[name] = load_data(name, cfg.DATA_DIRECTORY)

In [3]:
datasets_map['mnist']['train']['images'].shape

(60000, 28, 28)

# Construction des datasets

In [10]:
datasets_map['cifar']['train']['images'].shape

(150000, 32, 32)

In [9]:
cifar_train_dataset = tf.data.Dataset.from_tensor_slices((
    datasets_map['cifar']['train']['images'],
    datasets_map['cifar']['train']['labels']
))

2021-07-15 21:48:28.891313: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.


ValueError: Dimensions 150000 and 50000 are not compatible

In [8]:
cifar_train_dataset

<TensorDataset shapes: ((150000, 32, 32), (50000,)), types: (tf.float32, tf.int64)>

# Auto-encodeur

Nous commençons par un auto-encodeur en TensorFlow à l'aide d'une classe personnalisée.

In [ ]:
# Notre bloc encodeur
class Encoder(layers.Layer):
    def __init__(self, latent_size=8, *args, **kwargs):
        super(Encoder, self).__init__(*args, **kwargs)
        
        # Couches
        self.conv1 = layers.Conv2D(filters=4, kernel_size=3, strides=1, padding="SAME", use_bias=True)
        self.conv2 = layers.Conv2D(filters=1, kernel_size=3, strides=1, padding="SAME", use_bias=True)
        
        # Activations
        self.relu = layers.ReLu()
        
        # Couche latente
        self.dense = layers.Dense(units=128, use_bias=True)
        self.latent = layers.Dense(units=latent_size, use_bias=True)
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.dense(x)
        x = self.relu(x)
        return self.latent(x)

# Notre bloc décodeur
class Decoder(layers.Layer):
    def __init__(self, *args, **kwargs):
        super(Decoder, self).__init__(*args, **kwargs)
        
        # Couches
        self.de_conv1 = layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding="SAME", use_bias=True)
        self.de_conv2 = layers.Conv2DTranspose(filters=4, kernel_size=3, strides=1, padding="SAME", use_bias=True)
        
        # Activations
        self.relu = layers.ReLu()
        
        # Couche latente
        self.dense = layers.Dense(units=784, use_bias=True)
        self.de_latent = layers.Dense(units=128, use_bias=True)
    
    def call(self, inputs):
        x = self.de_latent(inputs)
        x = self.relu(x)
        x = self.dense(x)
        x = self.relu(x)
        x = self.de_conv2(x)
        x = self.relu(x)
        return self.de_conv1(x)


# Classe finale : combinaison encodeur - decodeur
class ConvolutionalAE(tf.keras.Model):
    def __init__(self, latent_size=8, *args, **kwargs):
        super(ConvolutionalAE, self).__init__(*args, **kwargs)
        self.encoder = Encoder(latent_size=latent_size)
        self.decoder = Decoder()

    def call(self, inputs):
        x = self.encoder(inputs)
        return self.decoder(x)

In [ ]:
lr = 0.01

model = ConvolutionalAE(latent_size=16)
loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)

Maintenant ce qu'il serait bien, ce serait de :
- Développer la fonction d'entrainement (les calculs réalisés lors d'une époque)
- Il semblerait qu'avec **tf.GradientTape()** nous avons la construction du graph avec les opérations effectuées durant le run
- Construire nos jeux de données séparément avec **tf.data.Dataset**
- Somme toute : continuer de suivre le tutoriel : https://www.tensorflow.org/tutorials/quickstart/advanced